# Sensor Based Activity Recoginition 
Challenge: cdl1 - Sensor based Activity Recognition  
Team: Lea Bütler, Manjavy Kirupa, Etienne Roulet, Si Ben Tran  

Aufgabe: Visualisierungen der gemessenen Sensordaten

Hier in diesem Notebook visualisieren wir alle gemessenen Sensordaten und versuchen so zu identifizieren, welche aufgenommenen Sensordaten von relevanz sein können bzw. welche widerum nicht. 

In [22]:
# Laden von Libraries
%pip install -r requirements.txt
from data_ingestion.pipeline_manager import PipelineRunner as PipelineRunner 
from data_ingestion import utilities as util
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt
import seaborn as snsb
import polars as pl
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


## Import von Daten

- Einlesen der Daten mittels Pipeline oder
- Einlesen der gefilterten Daten von 'all-sensormeasurement-plots'

In [23]:
# Daten einlesen
data = pd.read_csv('Alle_Messungen.csv', sep=',')
data

,Unnamed: 0,time,Accelerometer_x,Accelerometer_y,Accelerometer_z,Gyroscope_x,Gyroscope_y,Gyroscope_z,Magnetometer_x,Magnetometer_y,Magnetometer_z,Orientation_qx,Orientation_qy,Orientation_qz,id,user,class,id_combined
0,0,2023-02-27 15:02:04.548,0.000000,-3.552000,-9.140000,NaN,NaN,NaN,NaN,NaN,NaN,0.161765,-0.088290,-0.470671,01_SamsungA22-2023-02-27_15-02-03,Ben_Tran,Laufen,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen
1,1,2023-02-27 15:02:17.156,0.000000,9.722000,1.278000,NaN,NaN,NaN,NaN,NaN,NaN,-0.217750,-0.719579,0.631111,01_SamsungA22-2023-02-27_15-02-03,Ben_Tran,Laufen,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen
2,2,2023-02-27 15:02:31.574,-0.728000,9.779000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,-0.420650,0.568356,-0.598058,01_SamsungA22-2023-02-27_15-02-03,Ben_Tran,Laufen,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen
3,3,2023-02-27 15:02:16.685,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,43.500000,-13.143750,NaN,NaN,NaN,01_SamsungA22-2023-02-27_15-02-03,Ben_Tran,Laufen,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen
4,4,2023-02-27 15:02:32.295,NaN,NaN,NaN,NaN,NaN,NaN,-13.012501,0.000000,20.587502,NaN,NaN,NaN,01_SamsungA22-2023-02-27_15-02-03,Ben_Tran,Laufen,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2539528,2539528,2023-03-11 13:12:01.906,-0.165031,-0.047750,-0.002080,0.036409,0.031078,0.033035,-31.916840,-10.083267,-49.236946,0.120060,0.065630,0.988451,9_iphone13mini-2023-03-11_13-09-56,Weiping_Zhang,Velofahren,9_iphone13mini-2023-03-11_13-09-56Weiping_Zhan...
2539529,2539529,2023-03-11 13:12:01.916,-0.198710,-0.076285,0.047025,0.023099,0.046894,0.033637,-31.769516,-10.042038,-49.300545,0.119959,0.065806,0.988438,9_iphone13mini-2023-03-11_13-09-56,Weiping_Zhang,Velofahren,9_iphone13mini-2023-03-11_13-09-56Weiping_Zhan...
2539530,2539530,2023-03-11 13:12:01.926,-0.207555,-0.116692,0.068004,0.009885,0.053780,0.034982,-31.764618,-10.130127,-49.406891,0.119799,0.065911,0.988438,9_iphone13mini-2023-03-11_13-09-56,Weiping_Zhang,Velofahren,9_iphone13mini-2023-03-11_13-09-56Weiping_Zhan...
2539531,2539531,2023-03-11 13:12:01.936,-0.203920,-0.147709,0.035921,0.009365,0.045132,0.033097,-31.766403,-10.006668,-49.255692,0.119646,0.065983,0.988439,9_iphone13mini-2023-03-11_13-09-56,Weiping_Zhang,Velofahren,9_iphone13mini-2023-03-11_13-09-56Weiping_Zhan...


# NA-Werte behandeln

In [63]:
# Count of NA values per column
data.isna().sum()

Unnamed: 0             0
time                   0
Accelerometer_x    24219
Accelerometer_y    24219
Accelerometer_z    24219
Gyroscope_x         3420
Gyroscope_y         3420
Gyroscope_z         3420
Magnetometer_x     31374
Magnetometer_y     31374
Magnetometer_z     31374
Orientation_qx     31206
Orientation_qy     31206
Orientation_qz     31206
id                     0
user                   0
class                  0
id_combined            0
dtype: int64

In [64]:
# length of the dataframe
print('data_trimmed: ', len(data))

# na values in the dataframe
data.isna().sum()

# show NA values per user
data[data.isna().any(axis=1)].value_counts("user")

data_trimmed:  2539533


user
Ben_Tran        20325
Lea_Buetler     17317
Tobias_Buess     1098
dtype: int64

In [65]:
# show 5 sample rows with NA values
data_na = data[data.isna().any(axis=1)]

# group_by id_combined and select first col
na_file_list = list(data_na.groupby('id_combined').first().reset_index().id_combined)
display(na_file_list)

['01-SamsungA22-2023-02-27_14-54-28Ben_TranTreppenlaufen',
 '01_GalaxyS21Tobias_BuessLaufen',
 '01_GalaxyS21Tobias_BuessRennen',
 '01_GalaxyS21Tobias_BuessSitzen',
 '01_GalaxyS21Tobias_BuessTreppenlaufen',
 '01_GalaxyS21Tobias_BuessVelofahren',
 '01_Huawei_Fahrrad-2023-02-27_19-23-04Lea_BuetlerVelofahren',
 '01_Huawei_Laufen-2023-03-03_19-15-46Lea_BuetlerLaufen',
 '01_Huawei_Rennen-2023-03-15_17-22-07Lea_BuetlerRennen',
 '01_Huawei_Sitzen-2023-02-23_19-10-32Lea_BuetlerSitzen',
 '01_Huawei_Stehen-2023-03-01_14-27-19Lea_BuetlerStehen',
 '01_Huawei_Treppe-2023-03-01_14-59-00Lea_BuetlerTreppenlaufen',
 '01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen',
 '01_SamsungA22-2023-02-27_15-37-24Ben_TranVelofahren',
 '01_SamsungA22-2023-02-27_17-48-41Ben_TranSitzen',
 '01_SamsungA22-2023-02-28_07-17-05Ben_TranStehen',
 '01_SamsungA22-2023-03-01_17-06-47Ben_TranRennen',
 '02-SamsungA22-2023-02-27_14-57-04Ben_TranTreppenlaufen',
 '02-SamsungA22-2023-02-28_07-22-08Ben_TranStehen',
 '02_GalaxyS21Tobias

erklärung wieso NA = 0

In [68]:
# replace Na Values in Data with 0
data = data.fillna(0)

# Daten trimmen

In [69]:
# function to trim all files in all_ids
def trim_all_files(df, measurement_ids, cut_size):
    new_df = pd.DataFrame()
    for i in measurement_ids:
        file_i = df[df['id_combined'] == i].copy()
        file_i['time'] = pd.to_datetime(file_i['time'])
        time_min = file_i['time'].min()
        time_max = file_i['time'].max()
        time_min_trimmed = time_min + pd.Timedelta(seconds=cut_size)
        time_max_trimmed = time_max - pd.Timedelta(seconds=cut_size)
        file_i_trimmed = file_i[(file_i['time'] > time_min_trimmed) & (file_i['time'] < time_max_trimmed)]
        # add file to new df
        new_df = pd.concat([new_df, file_i_trimmed], axis=0)
    return new_df

# list of all unique values in id_combined
all_ids = data['id_combined'].unique()

# trim all files
data_trimmed = trim_all_files(data, all_ids, 5)
data_trimmed.head()

,Unnamed: 0,time,Accelerometer_x,Accelerometer_y,Accelerometer_z,Gyroscope_x,Gyroscope_y,Gyroscope_z,Magnetometer_x,Magnetometer_y,Magnetometer_z,Orientation_qx,Orientation_qy,Orientation_qz,id,user,class,id_combined
1,1,2023-02-27 15:02:17.156,0.000,9.722,1.278,0.0,0.0,0.0,0.000000,0.0,0.000000,-0.21775,-0.719579,0.631111,01_SamsungA22-2023-02-27_15-02-03,Ben_Tran,Laufen,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen
2,2,2023-02-27 15:02:31.574,-0.728,9.779,0.000,0.0,0.0,0.0,0.000000,0.0,0.000000,-0.42065,0.568356,-0.598058,01_SamsungA22-2023-02-27_15-02-03,Ben_Tran,Laufen,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen
3,3,2023-02-27 15:02:16.685,0.000,0.000,0.000,0.0,0.0,0.0,0.000000,43.5,-13.143750,0.00000,0.000000,0.000000,01_SamsungA22-2023-02-27_15-02-03,Ben_Tran,Laufen,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen
4,4,2023-02-27 15:02:32.295,0.000,0.000,0.000,0.0,0.0,0.0,-13.012501,0.0,20.587502,0.00000,0.000000,0.000000,01_SamsungA22-2023-02-27_15-02-03,Ben_Tran,Laufen,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen
5,5,2023-02-27 15:02:33.285,0.000,0.000,0.000,0.0,0.0,0.0,-22.612501,0.0,20.268751,0.00000,0.000000,0.000000,01_SamsungA22-2023-02-27_15-02-03,Ben_Tran,Laufen,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen


In [18]:
# export to csv
data_trimmed.to_csv('Alle_Messungen_trimmed.csv')